In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *
import folium
from folium.plugins import MiniMap
from geopy.geocoders import Nominatim
import time
import requests

In [2]:
house = pd.read_csv('./data/아파트 실거래가.csv')
house

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,등기일자,거래유형,중개사소재지
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,12,"220,000",4,1987,언주로 3,NaN,,중개거래,서울 강남구
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,21,"220,000",2,1987,언주로 3,NaN,,중개거래,서울 강남구
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202205,27,"216,000",2,1987,언주로 3,NaN,,중개거래,서울 강남구
3,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,102.32,202204,1,"369,000",13,2020,개포로 264,NaN,,중개거래,"서울 강남구, 서울 양천구"
4,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,136.06,202205,2,"420,000",17,2020,개포로 264,NaN,,중개거래,서울 강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202203,27,"73,000",20,1997,동일로 752,NaN,,중개거래,서울 중랑구
12681,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202207,20,"74,000",3,1997,동일로 752,NaN,,중개거래,서울 중랑구
12682,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),84.03,202207,27,"91,500",12,1997,동일로 752,NaN,,중개거래,서울 중랑구
12683,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),84.69,202204,9,"49,900",7,2003,동일로144길 74,NaN,,중개거래,서울 중랑구


In [3]:
house.drop(['번지', '본번', '부번','층', '계약년월', '계약일' , '층', '건축년도', '해제사유발생일', '등기일자', '거래유형', '중개사소재지'] ,axis=1, inplace=True)
house.columns

Index(['시군구', '단지명', '전용면적(㎡)', '거래금액(만원)', '도로명'], dtype='object')

In [5]:
house

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264
...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74


In [6]:
# # 데이터프레임 생성 (house 데이터프레임 예시)
# house_df = house

# # 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
# client_id = '5j924usqjd'
# client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# # '위도'와 '경도' 칼럼을 추가합니다.
# house_df['위도'] = None
# house_df['경도'] = None

# # 네이버 지오코딩 API를 호출하여 주소별 위도와 경도를 얻습니다.
# for index, row in house_df.iterrows():
#     address = row['도로명']
#     url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
#     headers = {
#         'X-NCP-APIGW-API-KEY-ID': client_id,
#         'X-NCP-APIGW-API-KEY': client_secret
#     }
#     response = requests.get(url, headers=headers)
    
#     if response.status_code == 200:
#         data = response.json()
#         if data.get('addresses'):
#             latitude = data['addresses'][0]['y']
#             longitude = data['addresses'][0]['x']
#             house_df.at[index, '위도'] = latitude
#             house_df.at[index, '경도'] = longitude

# # 결과를 출력합니다.
# house_df

In [9]:
house_df.drop(['위도', '경도'], axis=1, inplace=True)
house_df

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264
...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74


In [10]:
for index, row in house.iterrows():
    address = row['시군구']
    parts = address.split(' ')  # 공백으로 문자열 분할
    for part in parts:
        if '동' in part:  # '동'이 포함된 문자열 찾기
            house.at[index, '법정동'] = part
            break

# 결과를 출력합니다.
house


,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,법정동
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3,개포동
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264,개포동
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264,개포동
...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752,중화동
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752,중화동
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752,중화동
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74,중화동


In [11]:
house.drop(['도로명', '단지명'], axis=1, inplace=True)
house

,시군구,전용면적(㎡),거래금액(만원),법정동
0,서울특별시 강남구 개포동,79.97,"220,000",개포동
1,서울특별시 강남구 개포동,79.97,"220,000",개포동
2,서울특별시 강남구 개포동,79.97,"216,000",개포동
3,서울특별시 강남구 개포동,102.32,"369,000",개포동
4,서울특별시 강남구 개포동,136.06,"420,000",개포동
...,...,...,...,...
12680,서울특별시 중랑구 중화동,59.76,"73,000",중화동
12681,서울특별시 중랑구 중화동,59.76,"74,000",중화동
12682,서울특별시 중랑구 중화동,84.03,"91,500",중화동
12683,서울특별시 중랑구 중화동,84.69,"49,900",중화동


In [12]:
# '거래금액(만원)' 열에서 쉼표(,)를 제거하고 문자열을 숫자로 변환
house['거래금액(만원)'] = house['거래금액(만원)'].str.replace(',', '').astype(float)
house

,시군구,전용면적(㎡),거래금액(만원),법정동
0,서울특별시 강남구 개포동,79.97,220000.0,개포동
1,서울특별시 강남구 개포동,79.97,220000.0,개포동
2,서울특별시 강남구 개포동,79.97,216000.0,개포동
3,서울특별시 강남구 개포동,102.32,369000.0,개포동
4,서울특별시 강남구 개포동,136.06,420000.0,개포동
...,...,...,...,...
12680,서울특별시 중랑구 중화동,59.76,73000.0,중화동
12681,서울특별시 중랑구 중화동,59.76,74000.0,중화동
12682,서울특별시 중랑구 중화동,84.03,91500.0,중화동
12683,서울특별시 중랑구 중화동,84.69,49900.0,중화동


In [13]:
# '거래금액(만원)' 열에서 쉼표(,)를 제거하고 문자열을 숫자로 변환
house['전용면적(㎡)'] = house['전용면적(㎡)'].round()
house

,시군구,전용면적(㎡),거래금액(만원),법정동
0,서울특별시 강남구 개포동,80.0,220000.0,개포동
1,서울특별시 강남구 개포동,80.0,220000.0,개포동
2,서울특별시 강남구 개포동,80.0,216000.0,개포동
3,서울특별시 강남구 개포동,102.0,369000.0,개포동
4,서울특별시 강남구 개포동,136.0,420000.0,개포동
...,...,...,...,...
12680,서울특별시 중랑구 중화동,60.0,73000.0,중화동
12681,서울특별시 중랑구 중화동,60.0,74000.0,중화동
12682,서울특별시 중랑구 중화동,84.0,91500.0,중화동
12683,서울특별시 중랑구 중화동,85.0,49900.0,중화동


In [14]:
house['제곱미터당 가격(만원)'] = (house['거래금액(만원)'] / house['전용면적(㎡)']).round()
house

,시군구,전용면적(㎡),거래금액(만원),법정동,제곱미터당 가격(만원)
0,서울특별시 강남구 개포동,80.0,220000.0,개포동,2750.0
1,서울특별시 강남구 개포동,80.0,220000.0,개포동,2750.0
2,서울특별시 강남구 개포동,80.0,216000.0,개포동,2700.0
3,서울특별시 강남구 개포동,102.0,369000.0,개포동,3618.0
4,서울특별시 강남구 개포동,136.0,420000.0,개포동,3088.0
...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,60.0,73000.0,중화동,1217.0
12681,서울특별시 중랑구 중화동,60.0,74000.0,중화동,1233.0
12682,서울특별시 중랑구 중화동,84.0,91500.0,중화동,1089.0
12683,서울특별시 중랑구 중화동,85.0,49900.0,중화동,587.0


In [16]:
# '법정동'을 기준으로 그룹화하고 '제곱미터당 가격(만원)' 열을 내림차순으로 정렬
result = house.groupby('법정동')['제곱미터당 가격(만원)'].mean().reset_index()
result = result.sort_values(by='제곱미터당 가격(만원)', ascending=False)

# 결과 출력
result


,법정동,제곱미터당 가격(만원)
80,반포동,3752.153846
145,압구정동,3732.538462
193,잠원동,3154.351064
7,개포동,3027.757576
192,잠실동,2707.720497
...,...,...
114,성북동,551.000000
144,안암동5가,491.000000
185,이화동,435.000000
196,장충동2가,428.000000


In [23]:
count_df = pd.read_csv('./data/count_by_dong.csv')
count_df

,행정동명,학원 개수,면적,면적 당 학원 수,1제곱미터당 학원 수
0,가락1동,9,1340.0,0.0,0.006716
1,가락2동,22,960.0,0.0,0.022917
2,가락본동,14,1130.0,0.0,0.012389
3,가리봉동,1,400.0,0.0,0.002500
4,가산동,9,2520.0,0.0,0.003571
...,...,...,...,...,...
410,효창동,6,1120.0,0.0,0.005357
411,후암동,6,450.0,0.0,0.013333
412,휘경1동,3,530.0,0.0,0.005660
413,휘경2동,9,820.0,0.0,0.010976


In [25]:
# 데이터프레임 생성 (house 데이터프레임 예시)
house_df = house

# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 칼럼을 추가합니다.
count_df['위도'] = None
count_df['경도'] = None

# 네이버 지오코딩 API를 호출하여 주소별 위도와 경도를 얻습니다.
for index, row in count_df.iterrows():
    address = row['행정동명']
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('addresses'):
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            count_df.at[index, '위도'] = latitude
            count_df.at[index, '경도'] = longitude

# 결과를 출력합니다.
count_df

,행정동명,학원 개수,면적,면적 당 학원 수,1제곱미터당 학원 수,법정동,위도,경도
0,가락1동,9,1340.0,0.0,0.006716,None,37.4964898,127.1098653
1,가락2동,22,960.0,0.0,0.022917,None,37.4987054,127.1266714
2,가락본동,14,1130.0,0.0,0.012389,None,37.4955568,127.1217864
3,가리봉동,1,400.0,0.0,0.002500,None,37.4825557,126.8897489
4,가산동,9,2520.0,0.0,0.003571,None,37.4769094,126.8917326
...,...,...,...,...,...,...,...,...
410,효창동,6,1120.0,0.0,0.005357,None,37.5424569,126.9618498
411,후암동,6,450.0,0.0,0.013333,None,37.5486375,126.9781344
412,휘경1동,3,530.0,0.0,0.005660,None,37.592932,127.065725
413,휘경2동,9,820.0,0.0,0.010976,None,37.5902093,127.0685788


In [26]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in count_df.iterrows():
    latitude = row['위도']
    longitude = row['경도']
#     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                address = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                count_df.at[index, '법정동'] = address  # 주소 정보를 데이터프레임에 추가
count_df

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

,행정동명,학원 개수,면적,면적 당 학원 수,1제곱미터당 학원 수,법정동,위도,경도
0,가락1동,9,1340.0,0.0,0.006716,가락동,37.4964898,127.1098653
1,가락2동,22,960.0,0.0,0.022917,가락동,37.4987054,127.1266714
2,가락본동,14,1130.0,0.0,0.012389,가락동,37.4955568,127.1217864
3,가리봉동,1,400.0,0.0,0.002500,가리봉동,37.4825557,126.8897489
4,가산동,9,2520.0,0.0,0.003571,가산동,37.4769094,126.8917326
...,...,...,...,...,...,...,...,...
410,효창동,6,1120.0,0.0,0.005357,효창동,37.5424569,126.9618498
411,후암동,6,450.0,0.0,0.013333,후암동,37.5486375,126.9781344
412,휘경1동,3,530.0,0.0,0.005660,휘경동,37.592932,127.065725
413,휘경2동,9,820.0,0.0,0.010976,휘경동,37.5902093,127.0685788


In [27]:
# 결과 출력
count_df
count_df.to_csv('./data/법정동당 학원 개수.csv', index=False)

In [28]:
# '법정동' 열을 중심으로 그룹화하고 '학원 개수'와 '면적'을 합산합니다.
grouped_df = count_df.groupby('법정동').agg({'학원 개수': 'sum', '면적': 'sum'}).reset_index()

# 결과를 출력합니다.
grouped_df

,법정동,학원 개수,면적
0,가락동,45,3430.0
1,가리봉동,1,400.0
2,가산동,9,2520.0
3,가양동,19,6200.0
4,가회동,5,540.0
...,...,...,...
206,회기동,2,810.0
207,효창동,6,1120.0
208,후암동,6,450.0
209,휘경동,12,1350.0


In [29]:
grouped_df = grouped_df.sort_values(by='학원 개수', ascending=False)
grouped_df

,법정동,학원 개수,면적
44,대치동,701,3640.0
62,목동,327,4550.0
126,신정동,230,4290.0
176,중계동,194,9190.0
69,반포동,147,4940.0
...,...,...,...
89,삼청동,1,910.0
205,황학동,1,1050.0
157,응봉동,1,1170.0
1,가리봉동,1,400.0


In [30]:
grouped_df['1제곱킬로미터당 학원 수'] = (grouped_df['학원 개수'] / grouped_df['면적'] * 1000).round()
grouped_df

,법정동,학원 개수,면적,1제곱킬로미터당 학원 수
44,대치동,701,3640.0,193.0
62,목동,327,4550.0,72.0
126,신정동,230,4290.0,54.0
176,중계동,194,9190.0,21.0
69,반포동,147,4940.0,30.0
...,...,...,...,...
89,삼청동,1,910.0,1.0
205,황학동,1,1050.0,1.0
157,응봉동,1,1170.0,1.0
1,가리봉동,1,400.0,2.0


In [32]:
# '법정동'을 기준으로 그룹화하고 '제곱미터당 가격(만원)' 열을 내림차순으로 정렬한 데이터프레임 (result)
result = house.groupby('법정동')['제곱미터당 가격(만원)'].mean().reset_index()
result = result.sort_values(by='제곱미터당 가격(만원)', ascending=False)

# 'grouped_df'와 'result' 데이터프레임을 '법정동'을 기준으로 이너 조인
merged_df = pd.merge(grouped_df, result, on='법정동', how='inner')

# 결과 출력
merged_df


,법정동,학원 개수,면적,1제곱킬로미터당 학원 수,제곱미터당 가격(만원)
0,대치동,701,3640.0,193.0,2412.940678
1,목동,327,4550.0,72.0,1674.099099
2,신정동,230,4290.0,54.0,1497.466216
3,중계동,194,9190.0,21.0,1152.582192
4,반포동,147,4940.0,30.0,3752.153846
...,...,...,...,...,...
148,창신동,2,1650.0,1.0,1231.222222
149,이화동,2,1040.0,2.0,435.000000
150,수서동,1,790.0,1.0,2625.387097
151,황학동,1,1050.0,1.0,1020.810811


In [38]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 칼럼을 추가합니다.
merged_df['위도'] = None
merged_df['경도'] = None

# 네이버 지오코딩 API를 호출하여 주소별 위도와 경도를 얻습니다.
for index, row in merged_df.iterrows():
    address = row['법정동']
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('addresses'):
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            merged_df.at[index, '위도'] = latitude
            merged_df.at[index, '경도'] = longitude

# 결과를 출력합니다.
merged_df

,법정동,학원 개수,면적,1제곱킬로미터당 학원 수,제곱미터당 가격(만원),위도,경도
0,대치동,701,3640.0,193.0,2412.940678,37.4931821,127.0567047
1,목동,327,4550.0,72.0,1674.099099,37.5303777,126.8712323
2,신정동,230,4290.0,54.0,1497.466216,37.543392,126.930513
3,중계동,194,9190.0,21.0,1152.582192,37.6525986,127.0775275
4,반포동,147,4940.0,30.0,3752.153846,37.5021441,126.9868224
...,...,...,...,...,...,...,...
148,창신동,2,1650.0,1.0,1231.222222,37.570758,127.016363
149,이화동,2,1040.0,2.0,435.000000,37.577008,127.004338
150,수서동,1,790.0,1.0,2625.387097,37.4889181,127.104929
151,황학동,1,1050.0,1.0,1020.810811,37.5673446,127.0212812


In [55]:
merged_df_test = merged_df[:20]
merged_df_test

,법정동,학원 개수,면적,1제곱킬로미터당 학원 수,제곱미터당 가격(만원),위도,경도
0,대치동,701,3640.0,1.000000,0.596276,37.4931821,127.0567047
1,목동,327,4550.0,0.369792,0.373543,37.5303777,126.8712323
2,신정동,230,4290.0,0.276042,0.320295,37.543392,126.930513
3,중계동,194,9190.0,0.104167,0.216325,37.6525986,127.0775275
4,반포동,147,4940.0,0.151042,1.000000,37.5021441,126.9868224
5,서초동,139,6090.0,0.114583,0.365967,37.4901186,127.0195082
6,봉천동,111,12150.0,0.041667,0.197150,37.4779619,126.9534602
7,상계동,100,15530.0,0.026042,0.201365,37.6799006,127.0549781
8,화곡동,97,8730.0,0.052083,0.121503,37.544075,126.847725
9,방이동,97,3390.0,0.145833,0.352043,37.510933,127.123925


In [57]:
# 'merged_df' 데이터프레임에서 '법정동' 열을 기준으로 그룹화
grouped_df = merged_df.groupby('법정동')

# 각 그룹(법정동)에 대한 상관계수를 계산하고 데이터프레임에 추가
correlation_results = []

for 법정동, group_data in grouped_df:
    학원수열 = group_data['1제곱킬로미터당 학원 수']
    가격열 = group_data['제곱미터당 가격(만원)']
    상관계수 = 학원수열.corr(가격열)
    correlation_results.append({'법정동': 법정동, '상관계수': 상관계수})

# 결과를 데이터프레임으로 변환
correlation_df = pd.DataFrame(correlation_results)

# 데이터프레임에 상관계수가 추가된 것을 확인
print(correlation_df)


      법정동  상관계수
0     가락동   NaN
1    가리봉동   NaN
2     가산동   NaN
3     가양동   NaN
4     갈현동   NaN
..    ...   ...
148   화곡동   NaN
149   화양동   NaN
150   황학동   NaN
151   효창동   NaN
152   후암동   NaN

[153 rows x 2 columns]


In [58]:
# 'merged_df' 데이터프레임에서 '법정동' 열을 기준으로 그룹화
grouped_df = merged_df.groupby('법정동')

# 각 그룹(법정동)에 대한 상관계수를 계산하고 데이터프레임에 추가
correlation_results = []

for 법정동, group_data in grouped_df:
    학원수열 = group_data['1제곱킬로미터당 학원 수']
    가격열 = group_data['제곱미터당 가격(만원)']
    상관계수 = 학원수열.corr(가격열)
    correlation_results.append({'법정동': 법정동, '상관계수': 상관계수})

# 결과를 데이터프레임으로 변환
correlation_df = pd.DataFrame(correlation_results)

# 데이터프레임에 상관계수가 추가된 것을 확인
correlation_df


,법정동,상관계수
0,가락동,NaN
1,가리봉동,NaN
2,가산동,NaN
3,가양동,NaN
4,갈현동,NaN
...,...,...
148,화곡동,NaN
149,화양동,NaN
150,황학동,NaN
151,효창동,NaN


In [59]:
import numpy as np

# 1동과 2동의 데이터
가격 = np.array([1000, 2000])
학원수 = np.array([20, 25])

# 상관계수 계산
correlation = np.corrcoef(가격, 학원수)

print(f'1동과 2동의 가격과 학원 수 간 상관계수: {correlation[0, 1]}')


1동과 2동의 가격과 학원 수 간 상관계수: 0.9999999999999999


In [ ]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

house_df_test['주소'] = None

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in house_df_test.iterrows():
    latitude = row['위도']
    longitude = row['경도']
#     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        print(response)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                address = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                house_df_test.at[index, '주소'] = address  # 주소 정보를 데이터프레임에 추가
house_df_test

In [ ]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
# client_id = '5j924usqjd'
# client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# count_df['법정동'] = None

# # '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
# for index, row in count_df.iterrows():
#     latitude = row['위도']
#     longitude = row['경도']
# #     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
#     if latitude is not None and longitude is not None:
#         url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
#         headers = {
#             'X-NCP-APIGW-API-KEY-ID': client_id,
#             'X-NCP-APIGW-API-KEY': client_secret
#         }
#         response = requests.get(url, headers=headers)
#         if response.status_code == 200:
#             data = response.json()
#             if data.get('results'):
#                 address = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
#                 count_df.at[index, '주소'] = address  # 주소 정보를 데이터프레임에 추가
# count_df

In [ ]:
data = shop_seoul_cafe_concat
cafe_map = folium.Map(location=[data.위도.mean(), data.경도.mean()], zoom_start=12)

for index, cafe in data.iterrows():
    popup = folium.Popup('{}: {:,.0f}'.format(index, cafe.매장수), max_width=300)
    folium.CircleMarker(location=[cafe.위도, cafe.경도], popup=popup, radius=cafe.매장수 / 40, 
                        color='red',fill_color='blue').add_to(cafe_map)

cafe_map.save('./output/cafe_map.html')
cafe_map